In [240]:
#Binary Prediction

# Import pandas
import pandas as pd

# Import sklearn
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Import tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


# Load dataframe from dataset 
df = pd.read_csv('Input/Churn.csv')
df.head(1)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [241]:
# Dropping some unnecessary dtypes that doesn't affect the dataframe (Or does not have any pattern)
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [242]:
# Check to see the what the datatypes are numerical or not
df.info(verbose=True) # Geography and Gender are non num

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [243]:
# Convert 2 columns Geography and Gender into True and False
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


In [244]:
# Dropping Exited since it is label (output)
X = df.drop(columns=['Exited'])
y = df['Exited'].values # Return the value from y

# Splitting the data into Train set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




In [245]:
scaler = StandardScaler()

# Fit and then transform X_train, X_test
X_train_trf = scaler.fit_transform(X_train)
X_test_trf = scaler.fit_transform(X_test)

In [246]:
# Sigmoid Activation Function
model = Sequential([
    tf.keras.layers.Dense(units= 11, activation='sigmoid', input_dim=11),
    tf.keras.layers.Dense(units= 11, activation='sigmoid'),
    tf.keras.layers.Dense(units= 1, activation='sigmoid')
])

In [247]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 11)                132       
                                                                 
 dense_25 (Dense)            (None, 11)                132       
                                                                 
 dense_26 (Dense)            (None, 1)                 12        
                                                                 
Total params: 276 (1.08 KB)
Trainable params: 276 (1.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [248]:
#Assign optimizer, loss function and metrics
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [249]:
# Converting X_train and y_train to tensors
X_train_tensor = tf.convert_to_tensor(X_train_trf)
y_train_tensor = tf.convert_to_tensor(y_train)

X_test_tensor = tf.convert_to_tensor(X_test_trf)

# Fitting the model to reduce losses
model.fit(X_train_tensor, y_train_tensor, batch_size=50, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
128/128 [==============================] - 1s 2ms/step - loss: 0.6329 - accuracy: 0.6503 - val_loss: 0.5399 - val_accuracy: 0.7900
Epoch 2/100
128/128 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7961 - val_loss: 0.5039 - val_accuracy: 0.7900
Epoch 3/100
128/128 [==============================] - 0s 1ms/step - loss: 0.4893 - accuracy: 0.7961 - val_loss: 0.4945 - val_accuracy: 0.7900
Epoch 4/100
128/128 [==============================] - 0s 1ms/step - loss: 0.4806 - accuracy: 0.7961 - val_loss: 0.4870 - val_accuracy: 0.7900
Epoch 5/100
128/128 [==============================] - 0s 1ms/step - loss: 0.4731 - accuracy: 0.7961 - val_loss: 0.4798 - val_accuracy: 0.7900
Epoch 6/100
128/128 [==============================] - 0s 1ms/step - loss: 0.4661 - accuracy: 0.7961 - val_loss: 0.4729 - val_accuracy: 0.7900
Epoch 7/100
128/128 [==============================] - 0s 1ms/step - loss: 0.4593 - accuracy: 0.7961 - val_loss: 0.4659 - val_accuracy: 0.7900

In [250]:
# Get all y prediction based on test model tensors
y_pred = model.predict(X_test_tensor)
y_pred


63/63 [==============================] - 0s 771us/step


array([[0.05689274],
       [0.00877046],
       [0.1615263 ],
       ...,
       [0.2855632 ],
       [0.17448764],
       [0.16528569]], dtype=float32)

In [251]:
# Get maximum number with axis = -1 in all y prediction
y_pred = y_pred.argmax(axis=-1)

In [252]:
# Checking accuracy score
sk.metrics.accuracy_score(y_test, y_pred)

0.802